### Genderizer
Classificateur de genre : Utilisé dans le contexte de l'analyse linguistique ou du traitement automatique des langues.
Utiliser Pytorch


Feuille de route:
- faire un état de l'art rapide sur le web
    - Nooj
    - HuggingFace
- choisir un dictionaire gratuit open source > Dictionnaire de l’Académie française de 1932-35 (8e édition) - GLP3
- créer une base de données de noms communs étiquetés par genre
- faire une analyse des données

In [1]:
%pip install beautifulsoup4==4.12.3
%pip install pandas==2.2.3

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [3]:
# import os
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd
import urllib.request
import re
from IPython.display import display, HTML

In [8]:
# importer la source au format Mobipocket
!mkdir -p tmp
url = "https://www.ebooksgratuits.com/newsendbook.php?id=1609&format=mp"
urllib.request.urlretrieve(url, "tmp/aca.prc")

print("Download complete.")

645.61s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Download complete.


In [9]:
# installer le module mobi
%pip install mobi

676.32s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 2.2 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.9 MB/s eta 0:00:00
  Created wheel for mobi: filename=mobi-0.3.3-py3-none-any.whl size=101386 sha256=df0fa99583b0e2e0371847b214fc63cefe1a8ec1b00e474d7908aca56fabbc8d
  Stored in directory: /tmp/pip-ephem-wheel-cache-pptno5r6/wheels/ab/a5/f9/868c46bfcc147bde457dfe6fc77af286f2314e716471008bcb
Successfully built mobi
Note: you may need to restart the kernel to use updated packages.


In [10]:
# décompresser le fichier ~ 3 secondes
!mobiunpack -r tmp/aca.prc ./

775.86s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


KindleUnpack v0.82
   Based on initial mobipocket version Copyright © 2009 Charles M. Hannum <root@ihack.net>
   Extensive Extensions and Improvements Copyright © 2009-2014 
       by:  P. Durrant, K. Hendricks, S. Siebert, fandrieu, DiapDealer, nickredding, tkeo.
   This program is free software: you can redistribute it and/or modify
   it under the terms of the GNU General Public License as published by
   the Free Software Foundation, version 3.
Unpacking Book...
Completed


In [11]:
# créations de listes d'entrées du dictionnaire et de leurs définitions
# d'après les basises du fichier HTML
# prend environ 0,2 secondes

# Read the HTML content from the file
with open('./mobi7/book.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Define the regular expression patterns to match <idx:orth> and <idx:entry> tags
pattern_orth = re.compile(r'<idx:orth value="[^"]*">')
# pattern_entry = re.compile(r'<idx:entry>.*?</idx:entry>', re.DOTALL)
pattern_entry = re.compile(r'</idx:entry>.*?<mbp:pagebreak/>', re.DOTALL)

# Find all matches for <idx:orth> and <idx:entry> tags
orth_matches = pattern_orth.findall(html_content)
entry_matches = pattern_entry.findall(html_content)


In [12]:
# vérifier les nombres d'items trouvés
print("Number of <idx:orth> tags:", len(orth_matches))
print("Number of <idx:entry> tags:", len(entry_matches))

Number of <idx:orth> tags: 31933
Number of <idx:entry> tags: 31933


In [13]:
# Vérifier derniers items trouvés
# Print the last extracted tags
print("Extracted <idx:orth> tags:")
for tag in orth_matches[-4:]:
    print(tag)

print("\nExtracted <idx:entry> tags:")
for tag in entry_matches[-4:]:
    print(tag)

Extracted <idx:orth> tags:
<idx:orth value="ZOUAVE">
<idx:orth value="ZUT">
<idx:orth value="ZYGOMA">
<idx:orth value="ZYGOMATIQUE">

Extracted <idx:entry> tags:
</idx:entry>
<h2><font size="+2" color="BLUE"><b>ZOUAVE</b></font></h2><br/> <font color="green">n. m.</font><br/>Soldat d'un corps d'infanterie servant pour la plus grande partie en Afrique du Nord. <p> <i>Zouaves pontificaux,</i> Corps de volontaires constitué en 1860 par le général de Lamoricière pour la défense des États pontificaux. <mbp:pagebreak/>
</idx:entry>
<h2><font size="+2" color="BLUE"><b>ZUT</b></font></h2><br/> <font color="green">interj.</font><br/>Exclamation familière pour exprimer l'impatience, le dépit, pour envoyer les gens au diable et signifier qu'on ne veut plus rien entendre. <mbp:pagebreak/>
</idx:entry>
<h2><font size="+2" color="BLUE"><b>ZYGOMA</b></font></h2><br/> <font color="green">n. m.</font><br/><dom>T. d'Anatomie</dom><br/>Os de la pommette de la joue. <mbp:pagebreak/>
</idx:entry>
<h2><font

In [14]:
# crée la base de données acad_pocket.db
# et insère les mots et définitions trouvés d'après les listes orth_matches et entry_matches

db_file = "acad_pocket.db"         # SQLite database file

# === STEP 2: INITIALIZE SQLITE DATABASE ===
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Create a table for words and definitions
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dictionary (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        word TEXT,
        definition TEXT
    )
""")

# Insert words and definitions into the database
for word, definition in zip(orth_matches, entry_matches):

    word = re.search(r'value="([^"]*)"', word).group(1)
    definition = definition.replace('</idx:entry>', '').replace('<mbp:pagebreak/>', '')
    
    try:
        cursor.execute("INSERT INTO dictionary (word, definition) VALUES (?, ?)", (word, definition))
        
    except sqlite3.IntegrityError:
        pass  # Ignore duplicate words

# === STEP 4: FINALIZE ===
conn.commit()
conn.close()

print(f"Dictionary has been saved to {db_file}")

Dictionary has been saved to acad_pocket.db


In [15]:
# fonction pour afficher la ou les définitions d'un mot en texte brut et en HTML
def acad_definition(word):
    # Connect to the SQLite database
    conn = sqlite3.connect('acad_pocket.db')

    # Write a query to fetch data
    query = """
    SELECT * FROM dictionary
    WHERE word = '{}'
    """.format(word.upper())

    # Use pandas to read the data into a DataFrame
    df_ = pd.read_sql_query(query, conn)

    # Close the connection
    conn.close()

    # Check if the DataFrame is empty
    if df_.empty:
        print("Absent du dictionnaire de l'Académie Française de 1935")
    else:
        # Display the DataFrame with full column content
        pd.set_option('display.max_colwidth', None)
        display(df_)
        pd.reset_option('display.max_colwidth')
        for ht in df_.definition:
            # Display the HTML content
            display(HTML(ht))

    del df_, query, conn

In [16]:
acad_definition("tour")

id  word  \
0  30201  TOUR   
1  30202  TOUR   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

---

In [17]:
# liste les balises de book.html
# def list_tags(html_content):
#     # Parse the HTML content
#     soup = BeautifulSoup(html_content, 'html.parser')
#     list_balises = set()
#     # Iterate over all tags in the HTML
#     for tag in soup.find_all(True):
#         list_balises.add(tag.name)
#
#     return list_balises

In [20]:
# Read the HTML content from the file - environ 5 minutes
# with open('./mobi7/book.html', 'r', encoding='utf-8') as file:
#     html_content = file.read()
# print(list_tags(html_content))

In [17]:
# crée le dataframe acad basé sur la table dictionary de acad_pocket.db
with sqlite3.connect('acad_pocket.db') as conn:
    acad = pd.read_sql_query("SELECT * FROM dictionary", conn)
acad.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31933 entries, 0 to 31932
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          31933 non-null  int64 
 1   word        31933 non-null  object
 2   definition  31933 non-null  object
dtypes: int64(1), object(2)
memory usage: 748.6+ KB


In [18]:
# lister les balises vertes de la définition qui spécifient la ou les catégories grammaticales
# Add a new column to acad that includes the text between '<font color="green">' and '</font>' from the definition column
acad['cat_list'] = acad['definition'].apply(lambda x: re.findall(r'<font color="green">(.*?)</font>', x))

In [19]:
# première catégorie grammaticale trouvée parmi la liste
acad['cat'] = acad['cat_list'].apply(lambda x: x[0] if x else None)


In [20]:
# liste toutes les différentes catégories trouvées dans tous les mots
cat_list = []
for list in acad['cat_list']:
    if list:
        for item in list:
            if item not in cat_list:
                cat_list.append(item)
cat_list.sort()
print(cat_list)

['<i>(Je couvre ; nous couvrons. Je couvrais. Je couvris. Je couvrirai. Couvre. Que je couvre. Que je couvrisse. Couvrant. Couvert.)</i> v. tr.', 'Adjectif et pronom des deux genres', 'Adjectif possessif masculin', 'Adverbe', 'Conjonction', 'Conjonction alternative', 'Interj.', 'Interjection', 'Interjection familière', 'Interjection populaire', 'Interjection qui exprime la douleur.', 'Loc. adv.', 'Loc. adverb.', 'Loc. interj.', 'Loc. prép.', 'Locution', 'Locution adverbiale', 'Locution adverbiale empruntée du latin.', 'Locution conjonctive', 'Locution empruntée du latin.', 'Locution familière empruntée du latin.', 'Locution latine', 'Locution prépositive', 'Nom et adjectif de nombre cardinal invariable', 'Onomatopée', 'Participe présent', 'Participe présent employé substantivement', 'Particule', 'Particule adverbiale', 'Particule affirmative', "Particule d'origine latine", 'Particule grecque', 'Particule négative', 'Pron. dém.CELUI, m. sing; CELLE, f. sing.; CEUX, m. plur.; CELLES, f. 

In [21]:
# liste toutes les premières catégories trouvées dans tous les mots
cat_0_list = []
for item in acad['cat']:
    if item:
        if item not in cat_0_list:
            cat_0_list.append(item)
cat_0_list.sort()
print("\n".join(cat_0_list))

<i>(Je couvre ; nous couvrons. Je couvrais. Je couvris. Je couvrirai. Couvre. Que je couvre. Que je couvrisse. Couvrant. Couvert.)</i> v. tr.
Adjectif et pronom des deux genres
Adjectif possessif masculin
Adverbe
Conjonction
Conjonction alternative
Interj.
Interjection
Interjection familière
Interjection populaire
Interjection qui exprime la douleur.
Loc. adv.
Loc. adverb.
Loc. interj.
Loc. prép.
Locution
Locution adverbiale
Locution adverbiale empruntée du latin.
Locution empruntée du latin.
Locution familière empruntée du latin.
Locution latine
Nom et adjectif de nombre cardinal invariable
Onomatopée
Participe présent
Participe présent employé substantivement
Particule
Particule adverbiale
Particule affirmative
Particule d'origine latine
Particule grecque
Particule négative
Pron. dém.CELUI, m. sing; CELLE, f. sing.; CEUX, m. plur.; CELLES, f. plur.
Pronom de la troisième personne, des deux genres
Pronom invariable
Pronom masculin indéfini
Pronom personnel
Pronom personnel des deux ge

In [22]:
# catégories potentiellement associées à des noms communs

cat_noms = [ cat for cat in cat_0_list if 'n.' in cat.split() or 'nom' in cat.lower().split()]
print("Nombre de catégories de noms:", len(cat_noms))
print("\n".join(cat_noms))

Nombre de catégories de noms: 29
Nom et adjectif de nombre cardinal invariable
adj. des deux genres et n. m.
adj. et n.
adj. et n. des deux genres
adj. et n. f.
adj. et n. m.
adj. et nom numéral cardinal des deux genres
n.
n. des deux genres
n. et adj.
n. f.
n. f. coll.
n. f. et adj. des deux genres
n. f. et n. m.
n. f. pl.
n. f. pluriel
n. invariable
n. m.
n. m. coll.
n. m. et f.
n. m. et f. pl.
n. m. inv.
n. m. invariable
n. m. pl.
n. m. plur.
n. m. sans pluriel
n. m., Expression empruntée du latin.
n. m., Locution empruntée du latin.
nom et adj. numéral cardinal invariable


In [23]:
# Create a new DataFrame transco_cat with columns: cat, nom, nm, nf
transco_cat = pd.DataFrame(columns=['cat', 'nom', 'nm', 'nf'])

# Set the data types for the columns
transco_cat = transco_cat.astype({'cat': str, 'nom': bool, 'nm': bool, 'nf': bool})
transco_cat['cat'] = cat_noms
# Change all values in 'nom', 'nm', and 'nf' columns to False
transco_cat[['nom', 'nm', 'nf']] = False
# Update 'nom' column based on the condition
# acad['cat'] = acad['cat_list'].apply(lambda x: x[0] if x else None)
transco_cat['nom'] = transco_cat['cat'].apply(lambda x: True if ('n. m.' in x or 'n. f.' in x) else False)
transco_cat['nm'] = transco_cat['cat'].apply(lambda x: True if ('n. m.' in x) else False)
transco_cat['nf'] = transco_cat['cat'].apply(lambda x: True if ('n. f.' in x) else False)
transco_cat.to_csv('transco_cat.csv', index=False, sep='\t')


In [24]:
print(transco_cat)

                                              cat    nom     nm     nf
0   Nom et adjectif de nombre cardinal invariable  False  False  False
1                   adj. des deux genres et n. m.   True   True  False
2                                      adj. et n.  False  False  False
3                      adj. et n. des deux genres  False  False  False
4                                   adj. et n. f.   True  False   True
5                                   adj. et n. m.   True   True  False
6    adj. et nom numéral cardinal des deux genres  False  False  False
7                                              n.  False  False  False
8                              n. des deux genres  False  False  False
9                                      n. et adj.  False  False  False
10                                          n. f.   True  False   True
11                                    n. f. coll.   True  False   True
12                  n. f. et adj. des deux genres   True  False   True
13    

In [27]:
# correction des exceptions
for index, row in transco_cat.iterrows():
    cat = row['cat']
    if 'Nom et adjectif de nombre cardinal invariable' == cat: # HUIT
        transco_cat.loc[index, ['nom', 'nm', 'nf']] = [True, True, False]
        #
        # adj. et n. > 3 mots rares > ignorer
        # adj. et n. des deux genres > un seul mot rare > ignorer

    elif 'adj. et nom numéral cardinal des deux genres' == cat: # CENT
        transco_cat.loc[index, ['nom', 'nm', 'nf']] = [True, True, False]
        # 
        # n. > 799 mots dont 780 avec 2 terminaisons > à traiter
        # n. des deux genres > 223 mots > à traiter
    elif 'n. des deux genres' == cat: # CENT
        transco_cat.loc[index, ['nom', 'nm', 'nf']] = [True, True, True]

        # n. et adj. > GASCON
        # n. invariable > un seul mot rare PAT > ignorer
    elif 'nom et adj. numéral cardinal invariable' == cat: # MILLE
        transco_cat.loc[index, ['nom', 'nm', 'nf']] = [True, True, False]  


In [25]:
# lister les noms communs avec 2 formes de genre
# lister les terminaisons possibles
# Filter the DataFrame
filtered_df = acad[(acad['word'].str.contains(',')) & (acad['cat'] == 'n.')]
print(filtered_df.shape)
print(filtered_df)

(781, 5)
          id                   word  \
119      120     ABRÉVIATEUR, TRICE   
196      197       ACCAPAREUR, EUSE   
237      238  ACCOMPAGNATEUR, TRICE   
247      248            ACCORDÉ, ÉE   
264      265       ACCOUCHEUR, EUSE   
...      ...                    ...   
31690  31691        VIVANDIER, IÈRE   
31770  31771           VOLEUR, EUSE   
31782  31783      VOLTAIRIEN, IENNE   
31827  31828         VOYAGEUR, EUSE   
31896  31897        ZÉLATEUR, TRICE   

                                              definition cat_list cat  
119    \n<h2><font size="+2" color="BLUE"><b>ABRÉVIAT...     [n.]  n.  
196    \n<h2><font size="+2" color="BLUE"><b>ACCAPARE...     [n.]  n.  
237    \n<h2><font size="+2" color="BLUE"><b>ACCOMPAG...     [n.]  n.  
247    \n<h2><font size="+2" color="BLUE"><b>ACCORDÉ,...     [n.]  n.  
264    \n<h2><font size="+2" color="BLUE"><b>ACCOUCHE...     [n.]  n.  
...                                                  ...      ...  ..  
31690  \n<h2><font

In [26]:
# Find duplicates in the 'word' column
duplicates = filtered_df[filtered_df['word'].duplicated(keep=False)]

print(duplicates)

          id          word                                         definition  \
18491  18492  LOUEUR, EUSE  \n<h2><font size="+2" color="BLUE"><b>LOUEUR, ...   
18492  18493  LOUEUR, EUSE  \n<h2><font size="+2" color="BLUE"><b>LOUEUR, ...   

      cat_list cat  
18491     [n.]  n.  
18492     [n.]  n.  
